In [1]:
import pickle

with open('elastic_train_1000.bin','rb') as f:
    es_train = pickle.load(f)

In [2]:
import json

with open('../data/preprocess_wiki.json', "r", encoding="utf-8") as f:
    wiki = json.load(f)

In [3]:
len(list(es_train.values())[0])

1000

In [4]:
from datasets import load_from_disk

train = load_from_disk('../data/train_dataset')['train']
train[0]['document_id']
train_ground_truth_doc_ids = train['document_id']
len(train_ground_truth_doc_ids)

3952

In [5]:
train[0]

{'__index_level_0__': 42,
 'question': '대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?',
 'id': 'mrc-1-000067',
 'title': '미국 상원',
 'document_id': 18293,
 'answers': {'answer_start': [235], 'text': ['하원']},
 'context': '미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국 의회의 상원이다.\\n\\n미국 부통령이 상원의장이 된다. 각 주당 2명의 상원의원이 선출되어 100명의 상원의원으로 구성되어 있다. 임기는 6년이며, 2년마다 50개주 중 1/3씩 상원의원을 새로 선출하여 연방에 보낸다.\\n\\n미국 상원은 미국 하원과는 다르게 미국 대통령을 수반으로 하는 미국 연방 행정부에 각종 동의를 하는 기관이다. 하원이 세금과 경제에 대한 권한, 대통령을 포함한 대다수의 공무원을 파면할 권한을 갖고 있는 국민을 대표하는 기관인 반면 상원은 미국의 주를 대표한다. 즉 캘리포니아주, 일리노이주 같이 주 정부와 주 의회를 대표하는 기관이다. 그로 인하여 군대의 파병, 관료의 임명에 대한 동의, 외국 조약에 대한 승인 등 신속을 요하는 권한은 모두 상원에게만 있다. 그리고 하원에 대한 견제 역할(하원의 법안을 거부할 권한 등)을 담당한다. 2년의 임기로 인하여 급진적일 수밖에 없는 하원은 지나치게 급진적인 법안을 만들기 쉽다. 대표적인 예로 건강보험 개혁 당시 하원이 미국 연방 행정부에게 퍼블릭 옵션(공공건강보험기관)의 조항이 있는 반면 상원의 경우 하원안이 지나치게 세금이 많이 든다는 이유로 퍼블릭 옵션 조항을 제외하고 비영리건강보험기관이나 보험회사가 담당하도록 한 것이다. 이 경우처럼 상원은 하원이나 내각책임제가 빠지기 쉬운 국가들의 국회처럼 걸핏하면 발생하는 의회의 비정상적인 사태를 방지하는 기관이다. 상원은 급박한 처리사항의 경우가 아니면 법안을 먼저 내는 경우가 드물고 하원이 만든 법안

In [214]:
es_question = list(es_train.keys())
es_passage = list(es_train.values())

shuffles = []
for i, q in enumerate(es_question):
    gt_id = train_ground_truth_doc_ids[i]
    answer = train[i]['answers']['text'][0]

    concat_ids = [gt_id]
    for j, p_id in enumerate(es_passage[i]):
        if gt_id != p_id and p_id not in concat_ids and answer not in wiki[str(p_id)]:
                concat_ids.append(p_id)
        if len(concat_ids) == 5:
            break
    
    shuffles.append(concat_ids)

len(shuffles)

3952

In [215]:
shuffles[1029]

[58792, 58403, 56097, 56381, 54576]

In [94]:
shuffles_gt_0 = shuffles
shuffles_gt_1 = []
shuffles_gt_2 = []
shuffles_gt_3 = []
shuffles_gt_4 = []

for i in range(len(shuffles)):
    shuffle_gt_1 = shuffles[i].copy()
    shuffle_gt_1[0], shuffle_gt_1[1] = shuffle_gt_1[1], shuffle_gt_1[0]

    shuffle_gt_2 = shuffles[i].copy()
    tmp_gt = shuffle_gt_2.pop(0)
    shuffle_gt_2.insert(2, tmp_gt)

    shuffle_gt_3 = shuffles[i].copy()
    tmp_gt = shuffle_gt_3.pop(0)
    shuffle_gt_3.insert(3, tmp_gt)

    shuffle_gt_4 = shuffles[i].copy()
    tmp_gt = shuffle_gt_4.pop(0)
    shuffle_gt_4.append(tmp_gt)

    shuffles_gt_1.append(shuffle_gt_1)
    shuffles_gt_2.append(shuffle_gt_2)
    shuffles_gt_3.append(shuffle_gt_3)
    shuffles_gt_4.append(shuffle_gt_4)

In [95]:
print(shuffles_gt_0[0])
print(shuffles_gt_1[0])
print(shuffles_gt_2[0])
print(shuffles_gt_3[0])
print(shuffles_gt_4[0])

[18293, 10598, 6837, 23901, 42901]
[10598, 18293, 6837, 23901, 42901]
[10598, 6837, 18293, 23901, 42901]
[10598, 6837, 23901, 18293, 42901]
[10598, 6837, 23901, 42901, 18293]


In [96]:
len(shuffles)

3952

In [97]:
len(wiki[str(shuffles_gt_1[0][:1][0])]['text'])

561

In [168]:
shuffles_gt_1_answers = {}
shuffles_gt_2_answers = {}
shuffles_gt_3_answers = {}
shuffles_gt_4_answers = {}

for i in range(len(shuffles)):
    answers = train[i]['answers']
    passage_len = 0
    for concat_wiki_id in shuffles_gt_1[i][:1]:
        passage_len += len(wiki[str(concat_wiki_id)]['text'])
    answers['answer_start'] = [passage_len + 1 + answers['answer_start'][0]] # insert one space when concatenate
    shuffles_gt_1_answers[i] = answers

    answers = train[i]['answers']
    passage_len = 0
    for concat_wiki_id in shuffles_gt_2[i][:2]:
        passage_len += len(wiki[str(concat_wiki_id)]['text'])
    answers['answer_start'] = [passage_len + 2 + answers['answer_start'][0]] # insert two space when concatenate
    shuffles_gt_2_answers[i] = answers

    answers = train[i]['answers']
    passage_len = 0
    for concat_wiki_id in shuffles_gt_3[i][:3]:
        passage_len += len(wiki[str(concat_wiki_id)]['text'])
    answers['answer_start'] = [passage_len + 3 + answers['answer_start'][0]] # insert three space when concatenate
    shuffles_gt_3_answers[i] = answers

    answers = train[i]['answers']
    passage_len = 0
    for concat_wiki_id in shuffles_gt_4[i][:4]:
        passage_len += len(wiki[str(concat_wiki_id)]['text'])
    answers['answer_start'] = [passage_len + 4 + answers['answer_start'][0]] # insert four space when concatenate
    shuffles_gt_4_answers[i] = answers

In [147]:
print(shuffles_gt_1_answers[0])
print(shuffles_gt_2_answers[0])
print(shuffles_gt_3_answers[0])
print(shuffles_gt_4_answers[0])

{'answer_start': [562], 'text': ['하원']}
{'answer_start': [984], 'text': ['하원']}
{'answer_start': [1750], 'text': ['하원']}
{'answer_start': [2121], 'text': ['하원']}


In [169]:
import pandas as pd

# new_train = load_from_disk('../data/new_train_dataset')['train']

train_df = pd.DataFrame(train)

concat_train_df_gt_0 = train_df.copy()
concat_train_df_gt_1 = train_df.copy()
concat_train_df_gt_2 = train_df.copy()
concat_train_df_gt_3 = train_df.copy()
concat_train_df_gt_4 = train_df.copy()

In [101]:
with open('../data/wikipedia_documents.json', "r", encoding="utf-8") as f:
    origin_wiki = json.load(f)

In [170]:
contexts = []
for i, row in concat_train_df_gt_0.iterrows():
    context = origin_wiki[str(shuffles_gt_0[i][0])]['text']
    for j in range(1, len(shuffles_gt_0[i])):
        context += ' ' + wiki[str(shuffles_gt_0[i][j])]['text']
    contexts.append(context)
concat_train_df_gt_0['context'] = contexts

contexts = []
for i, row in concat_train_df_gt_1.iterrows():
    context = wiki[str(shuffles_gt_1[i][0])]['text']
    context += ' ' + origin_wiki[str(shuffles_gt_1[i][1])]['text']
    for j in range(2, len(shuffles_gt_1[i])):
        context += ' ' + wiki[str(shuffles_gt_1[i][j])]['text']
    contexts.append(context)
concat_train_df_gt_1['context'] = contexts
concat_train_df_gt_1['answers'] = shuffles_gt_1_answers.values()

contexts = []
for i, row in concat_train_df_gt_2.iterrows():
    context = wiki[str(shuffles_gt_2[i][0])]['text']
    context += ' ' + wiki[str(shuffles_gt_2[i][1])]['text']
    context += ' ' + origin_wiki[str(shuffles_gt_2[i][2])]['text']
    for j in range(3, len(shuffles_gt_2[i])):
        context += ' ' + wiki[str(shuffles_gt_2[i][j])]['text']
    contexts.append(context)
concat_train_df_gt_2['context'] = contexts
concat_train_df_gt_2['answers'] = shuffles_gt_2_answers.values()

contexts = []
for i, row in concat_train_df_gt_3.iterrows():
    context = wiki[str(shuffles_gt_3[i][0])]['text']
    context += ' ' + wiki[str(shuffles_gt_3[i][1])]['text']
    context += ' ' + wiki[str(shuffles_gt_3[i][2])]['text']
    context += ' ' + origin_wiki[str(shuffles_gt_3[i][3])]['text']
    for j in range(4, len(shuffles_gt_3[i])):
        context += ' ' + wiki[str(shuffles_gt_3[i][j])]['text']
    contexts.append(context)
concat_train_df_gt_3['context'] = contexts
concat_train_df_gt_3['answers'] = shuffles_gt_3_answers.values()

contexts = []
for i, row in concat_train_df_gt_4.iterrows():
    context = wiki[str(shuffles_gt_4[i][0])]['text']
    context += ' ' + wiki[str(shuffles_gt_4[i][1])]['text']
    context += ' ' + wiki[str(shuffles_gt_4[i][2])]['text']
    context += ' ' + wiki[str(shuffles_gt_4[i][3])]['text']
    context += ' ' + origin_wiki[str(shuffles_gt_4[i][4])]['text']
    contexts.append(context)
concat_train_df_gt_4['context'] = contexts
concat_train_df_gt_4['answers'] = shuffles_gt_4_answers.values()

In [175]:
for i, row in concat_train_df_gt_0.iterrows():
    if row['context'][row['answers']['answer_start'][0]] != row['answers']['text'][0][0]:
        print(i)
for i, row in concat_train_df_gt_1.iterrows():
    if row['context'][row['answers']['answer_start'][0]] != row['answers']['text'][0][0]:
        print(i)
for i, row in concat_train_df_gt_2.iterrows():
    if row['context'][row['answers']['answer_start'][0]] != row['answers']['text'][0][0]:
        print(i)
for i, row in concat_train_df_gt_3.iterrows():
    if row['context'][row['answers']['answer_start'][0]] != row['answers']['text'][0][0]:
        print(i)
for i, row in concat_train_df_gt_4.iterrows():
    if row['context'][row['answers']['answer_start'][0]] != row['answers']['text'][0][0]:
        print(i)

In [174]:
from datasets import Dataset

Dataset.from_pandas(concat_train_df_gt_0).save_to_disk("../passage_shuffle/concat_train_gt_0")
Dataset.from_pandas(concat_train_df_gt_1).save_to_disk("../passage_shuffle/concat_train_gt_1")
Dataset.from_pandas(concat_train_df_gt_2).save_to_disk("../passage_shuffle/concat_train_gt_2")
Dataset.from_pandas(concat_train_df_gt_3).save_to_disk("../passage_shuffle/concat_train_gt_3")
Dataset.from_pandas(concat_train_df_gt_4).save_to_disk("../passage_shuffle/concat_train_gt_4")

In [178]:
concat_train_df_gt_0.iloc[0]

title                                                            미국 상원
context              미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...
question                              대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?
id                                                        mrc-1-000067
answers                        {'answer_start': [235], 'text': ['하원']}
document_id                                                      18293
__index_level_0__                                                   42
Name: 0, dtype: object

In [180]:
shuffles_gt_0[0]

[18293, 10598, 6837, 23901, 42901]

In [ ]:
fids = train_df.copy()

ctxs = []
for i, row in fids.iterrows():
    ctx = [
        {
            'title': row['title'],
            'text': origin_wiki[str(shuffles_gt_0[i][0])]['text']
        }
    ]
    for j in range(1, len(shuffles_gt_0[i])):
        ctx.append(
            {
                'title': wiki[str(shuffles_gt_0[i][j])]['title'],
                'text': wiki[str(shuffles_gt_0[i][j])]['text']
            }
        )
    ctxs.append(ctx)

fids['ctxs'] = ctxs

answers = []
targets = []
for i, row in fids.iterrows():
    targets.append(row['answers']['text'][0])
    answers.append([targets[i]])
fids['target'] = targets
fids['answers'] = answers

fids

In [194]:
fids.drop(columns=['context'])
Dataset.from_pandas(fids).save_to_disk('../fids_dataset')

In [ ]:
vd = load_from_disk('../data/train_dataset')['validation']
vd

fid_vd = pd.DataFrame(vd)

ctxs = []
for i, row in fid_vd.iterrows():
    ctx = [
        {
            'title': row['title'],
            'text': origin_wiki[str(row['document_id'])]['text']
        }
    ]
    ctxs.append(ctx)

fid_vd['ctxs'] = ctxs

answers = []
targets = []
for i, row in fid_vd.iterrows():
    targets.append(row['answers']['text'][0])
    answers.append([targets[i]])
fid_vd['target'] = targets
fid_vd['answers'] = answers
fid_vd.drop(columns=['context'])
fid_vd

In [197]:
Dataset.from_pandas(fid_vd).save_to_disk('../fids_dataset_valid')

In [ ]:
es_question = list(es_train.keys())
es_passage = list(es_train.values())

shuffles = []
for i, q in enumerate(es_question):
    gt_id = train_ground_truth_doc_ids[i]
    answer = train[i]['answers']['text'][0]

    concat_ids = [gt_id]
    for j, p_id in enumerate(es_passage[i]):
        if gt_id != p_id and p_id not in concat_ids and answer not in wiki[str(p_id)]:
                concat_ids.append(p_id)
        if len(concat_ids) == 5:
            break
    
    shuffles.append(concat_ids)

len(shuffles)

In [199]:
len(shuffles)

3952

In [200]:
shuffles[0]

[18293, 10598, 6837, 23901, 42901]

In [216]:
random_shuffles = [shuffle for shuffle in shuffles]

gt_passage_table = {}
gt_passage_table = {i: concat_ids[0] for i, concat_ids in enumerate(random_shuffles)}
gt_passage_table[0]

18293

In [217]:
import random

random.seed(42)

for i in range(len(random_shuffles)):
    random.shuffle(random_shuffles[i])
random_shuffles[0]

[23901, 10598, 6837, 42901, 18293]

In [219]:
random_shuffles[1], gt_passage_table[1]

([51632, 51631, 51638, 51633, 51630], 51638)

In [223]:
random_shuffles_answers = {}

for i in range(len(random_shuffles)):
    answers = train[i]['answers']
    passage_len = 0

    if random_shuffles[i].index(gt_passage_table[i]) == 0:
        pass

    elif random_shuffles[i].index(gt_passage_table[i]) == 1:
        for concat_wiki_id in random_shuffles[i][:1]:
            passage_len += len(wiki[str(concat_wiki_id)]['text'])
        answers['answer_start'] = [passage_len + 1 + answers['answer_start'][0]] # insert one space when concatenate

    elif random_shuffles[i].index(gt_passage_table[i]) == 2:
        for concat_wiki_id in random_shuffles[i][:2]:
            passage_len += len(wiki[str(concat_wiki_id)]['text'])
        answers['answer_start'] = [passage_len + 2 + answers['answer_start'][0]] # insert two space when concatenate

    elif random_shuffles[i].index(gt_passage_table[i]) == 3:
        for concat_wiki_id in random_shuffles[i][:3]:
            passage_len += len(wiki[str(concat_wiki_id)]['text'])
        answers['answer_start'] = [passage_len + 3 + answers['answer_start'][0]] # insert three space when concatenate

    elif random_shuffles[i].index(gt_passage_table[i]) == 4:
        for concat_wiki_id in random_shuffles[i][:4]:
            passage_len += len(wiki[str(concat_wiki_id)]['text'])
        answers['answer_start'] = [passage_len + 4 + answers['answer_start'][0]] # insert four space when concatenate
        
    random_shuffles_answers[i] = answers

In [225]:
random_shuffles_answers[0]

{'answer_start': [2356], 'text': ['하원']}

In [226]:
train_df = pd.DataFrame(train)

random_shuffles_df = train_df.copy()

In [227]:
contexts = []

for i, row in random_shuffles_df.iterrows():
    if random_shuffles[i].index(gt_passage_table[i]) == 0:
        context = origin_wiki[str(random_shuffles[i][0])]['text']
        for j in range(1, len(random_shuffles[i])):
            context += ' ' + wiki[str(random_shuffles[i][j])]['text']
        contexts.append(context)
    elif random_shuffles[i].index(gt_passage_table[i]) == 1:
        context = wiki[str(random_shuffles[i][0])]['text']
        context += ' ' + origin_wiki[str(random_shuffles[i][1])]['text']
        for j in range(2, len(random_shuffles[i])):
            context += ' ' + wiki[str(random_shuffles[i][j])]['text']
        contexts.append(context)
    elif random_shuffles[i].index(gt_passage_table[i]) == 2:
        context = wiki[str(random_shuffles[i][0])]['text']
        context += ' ' + wiki[str(random_shuffles[i][1])]['text']
        context += ' ' + origin_wiki[str(random_shuffles[i][2])]['text']
        for j in range(3, len(random_shuffles[i])):
            context += ' ' + wiki[str(random_shuffles[i][j])]['text']
        contexts.append(context)
    elif random_shuffles[i].index(gt_passage_table[i]) == 3:
        context = wiki[str(random_shuffles[i][0])]['text']
        context += ' ' + wiki[str(random_shuffles[i][1])]['text']
        context += ' ' + wiki[str(random_shuffles[i][2])]['text']
        context += ' ' + origin_wiki[str(random_shuffles[i][3])]['text']
        for j in range(4, len(random_shuffles[i])):
            context += ' ' + wiki[str(random_shuffles[i][j])]['text']
        contexts.append(context)
    elif random_shuffles[i].index(gt_passage_table[i]) == 4:
        context = wiki[str(random_shuffles[i][0])]['text']
        context += ' ' + wiki[str(random_shuffles[i][1])]['text']
        context += ' ' + wiki[str(random_shuffles[i][2])]['text']
        context += ' ' + wiki[str(random_shuffles[i][3])]['text']
        context += ' ' + origin_wiki[str(random_shuffles[i][4])]['text']
        contexts.append(context)

random_shuffles_df['context'] = contexts
random_shuffles_df['answers'] = random_shuffles_answers.values()
random_shuffles_df

,title,context,question,id,answers,document_id,__index_level_0__
0,미국 상원,"미국의 외교정책의 수립과 이행에 대한 주된 책임을 대통령이 지게 되어 있으나, 의회...",대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,mrc-1-000067,"{'answer_start': [2356], 'text': ['하원']}",18293,42
1,인사조직관리,인류 역사에서 무리를 지어 살기 시작하면서 조직이 시작되었다. 인류를 위한 다양한 ...,현대적 인사조직관리의 시발점이 된 책은?,mrc-0-004397,"{'answer_start': [1287], 'text': ['《경영의 실제》']}",51638,2873
2,강희제,"봉서재(鳳棲齋)는 조선시대 중기 결성장씨(結城張氏)인 장원주(張元胄, 1634~17...",강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,mrc-1-000362,"{'answer_start': [3987], 'text': ['백성']}",5028,230
3,금동삼존불감,이 불상군은 1978년 발견되어 학계에 알려졌으며 봉황리 햇골산 중턱 두 곳에 약간...,11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,mrc-0-001510,"{'answer_start': [8962], 'text': ['중국']}",34146,992
4,계사명 사리구,소총통은 조선 중기의 대표적인 소형총통인 승자총통의 발전된 형식이다. 약통에 도화선...,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,mrc-0-000823,"{'answer_start': [2958], 'text': ['4개']}",47334,548
...,...,...,...,...,...,...,...
3947,이오 (위성),이오의 산\n이오의 산 목록\n 이오에는 100~150개의 산이 있다. 이들 산의 ...,대부분의 이오의 산이 형성되는데 영향을 끼친 것은?,mrc-0-005285,"{'answer_start': [317], 'text': ['지질 구조']}",8275,3445
3948,내니 다이어리,1933년 뉴욕 시립 고아원. 애니에게는 11년 후 찾아오겠다는 부모의 편지와 목걸...,애니는 어디서 태어났는가?,mrc-1-000699,"{'answer_start': [1005], 'text': ['뉴저지']}",37764,450
3949,빈 필하모니 관현악단,"1925년에 덴마크 방송 협회(DR) 발족과 함께 부속 악단으로 창단되었으며, 그 ...",1854년에 니콜라이의 뒤를 이어 상임 지휘자로서 활동한 인물은?,mrc-0-003429,"{'answer_start': [4898], 'text': ['칼 에케르트']}",12730,2252
3950,드래곤,델포이의 신탁에 따라 암소를 따라간 카드모스는 테베 땅에 이르렀다. 카드모스는 암소...,드래곤의 암수 구분이 있는 신화는?,mrc-0-003956,"{'answer_start': [2516], 'text': ['슬라브 신화']}",10924,2595


In [229]:
for i, row in random_shuffles_df.iterrows():
    if row['context'][row['answers']['answer_start'][0]] != row['answers']['text'][0][0]:
        print(i)

In [230]:
Dataset.from_pandas(random_shuffles_df).save_to_disk('../passage_random_shuffle')

In [256]:
qg = pd.read_csv('../wiki_title_question_generation.csv')
qg

,Unnamed: 0,answers,context,id,question
0,0,"{'text': ['나라 목록'], 'answer_start': [6]}","이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개...",mrc-qg-title-0,전 세계 206개 나라의 현황과 주권 승인 정보를 개요 형태로 나열하고 있는 문서는...
1,1,"{'text': ['백남준'], 'answer_start': [432]}",현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이다. 친...,mrc-qg-title-1,1966년 졸업과 함께 독일로 유학을 떠나 뮌헨 대학교 및 쾰른 대학교 등에서 서양...
2,2,"{'text': ['아오조라 문고'], 'answer_start': [0]}","아오조라 문고(靑空文庫, あおぞらぶんこ|아오조라 분고)는 ‘일본어판 구텐베르크 프로...",mrc-qg-title-2,일본어판 구텐베르크 프로젝트로 불리는 일본의 인터넷 전자도서관이 어디지?
3,3,"{'text': ['데니스 리치'], 'answer_start': [426]}",켄 톰슨(Ken Thompson) 등과 함께 최초의 유닉스(Unix) 시스템을 개발...,mrc-qg-title-3,현재 애플의 컴퓨터는 대부분 누구의 업적을 따르고 있어?
4,4,"{'text': ['주기율표'], 'answer_start': [0]}","주기율표(週期律表, 주기률표, periodic table) 또는 주기표(週期表)는 ...",mrc-qg-title-4,원소를 구분하기 쉽게 성질에 따라 배열한 표가 뭐야?
...,...,...,...,...,...
17334,17334,"{'text': ['타바투이호'], 'answer_start': [0]}",타바투이호의 형성과 관련하여 전해지는 전설이 있다. 예전에 이 지역에는 아름답고 강...,mrc-qg-title-17334,"어느 호수의 형성과 관련하여 전해지는 전설이 있는데, 이 호수의 이름은 무엇인가?"
17335,17335,"{'text': ['2022년 FIFA 월드컵 유럽 지역 예선 플레이오프'], 'a...",2022년 FIFA 월드컵 유럽 지역 예선 플레이오프 진행 방식은 2019년 12월...,mrc-qg-title-17335,유럽 축구 연맹 집행위원회를 통해 확정된 진행 방식은?
17336,17336,"{'text': ['자주국방론'], 'answer_start': [8]}","박정희 정부가 자주국방론을 내놓은 배경은 ""아시아의 공산주의 국가와 자유진영 국가의...",mrc-qg-title-17336,"박정희 정부가 아시아의 공산주의 국가와 자유진영 국가의 분쟁이 발생했을 시, 분쟁의..."
17337,17337,"{'text': ['오키나와 현립 박물관·미술관'], 'answer_start': ...","오키나와 현립 박물관·미술관( , Okinawa Prefectural Museum ...",mrc-qg-title-17337,오키나와에 있는 현립 최초의 복합 문화 시설은?


In [283]:
with open('../es_qg_title.pickle','rb') as f:
    es_qg = pickle.load(f)

In [284]:
from tqdm import tqdm
qg_passage_id_map = {}
for i, row in tqdm(qg.iterrows(), total=len(qg)):
    c = row['context']
    for j in range(len(wiki)):
        if wiki[str(j)]['text'] == c:
            qg_passage_id_map[i] = j
len(qg_passage_id_map)

100%|██████████| 17339/17339 [11:20<00:00, 25.47it/s]


17339

In [285]:
err_idx = [266,378,407,636,712,860,890,947,1030,1106,1202,1608,1693,1764,1895,2082,2084,2232,2238,2468,2502,2554,2565,2870,2954,3253,3308,3399,3416,3632,3700,4025,4483,4623,4718,4882,5338,5435,5542,5633,5755,5983,6255,6279,6351,6575,6725,6969,7211,7574,7937,8193,8486,8719,8760,8798,9048,9190,9462,9594,9671,10239,10752,10849,10927,11178,11482,11693,11937,11977,11981,12111,12334,12596,12612,12677,12772,12841,12842,12853,12932,13061,13090,13157,13264,13302,13414,13424,13641,13929,14019,14044,14189,14230,14812,14899,15004,15035,15041,15087,15145,15292,15311,15327,15371,15382,15914,16014,16270,16275,16615,16652,16845,17115,17203,17306]
len(err_idx)

116

In [277]:
qg_question = list(es_qg.keys())
qg_passage = list(es_qg.values())

qg_train = []
errs = []
for i, q in enumerate(qg_question):
    gt_id = qg_passage_id_map[i]
    answer = eval(qg['answers'][i])['text'][0]

    concat_ids = [gt_id]
    for j, p_id_score in enumerate(qg_passage[i]):
        p_id = p_id_score['id']
        if gt_id != p_id and p_id not in concat_ids and answer not in wiki[p_id]:
            concat_ids.append(p_id)
        if len(concat_ids) == 5:
            break
        if j == len(qg_passage[i]) - 1 and len(concat_ids) < 5:
            errs.append(i)
    
    qg_train.append(concat_ids)

len(qg_train)

17222

In [280]:
len(qg_passage)

17222

In [278]:
errs

[]

In [265]:
qg_train[0]

[4554, '2', '4556', '12322', '49426']

In [266]:
qg_random_shuffles = [qg_ids for qg_ids in qg_train]

qg_gt_passage_table = {}
qg_gt_passage_table = {i: concat_ids[0] for i, concat_ids in enumerate(qg_random_shuffles)}
qg_gt_passage_table[0]

4554

In [270]:
import random

random.seed(42)

for i in range(len(qg_random_shuffles)):
    random.shuffle(qg_random_shuffles[i])

qg_random_shuffles_answers = {}

for i in range(len(qg_random_shuffles)):
    answers = eval(qg['answers'][i])
    passage_len = 0

    if qg_random_shuffles[i].index(qg_gt_passage_table[i]) == 0:
        pass

    elif qg_random_shuffles[i].index(qg_gt_passage_table[i]) == 1:
        for concat_wiki_id in qg_random_shuffles[i][:1]:
            passage_len += len(wiki[str(concat_wiki_id)]['text'])
        answers['answer_start'] = [passage_len + 1 + answers['answer_start'][0]] # insert one space when concatenate

    elif qg_random_shuffles[i].index(qg_gt_passage_table[i]) == 2:
        for concat_wiki_id in qg_random_shuffles[i][:2]:
            passage_len += len(wiki[str(concat_wiki_id)]['text'])
        answers['answer_start'] = [passage_len + 2 + answers['answer_start'][0]] # insert two space when concatenate

    elif qg_random_shuffles[i].index(qg_gt_passage_table[i]) == 3:
        for concat_wiki_id in qg_random_shuffles[i][:3]:
            passage_len += len(wiki[str(concat_wiki_id)]['text'])
        answers['answer_start'] = [passage_len + 3 + answers['answer_start'][0]] # insert three space when concatenate

    elif qg_random_shuffles[i].index(qg_gt_passage_table[i]) == 4:
        for concat_wiki_id in qg_random_shuffles[i][:4]:
            passage_len += len(wiki[str(concat_wiki_id)]['text'])
        answers['answer_start'] = [passage_len + 4 + answers['answer_start'][0]] # insert four space when concatenate
        
    qg_random_shuffles_answers[i] = answers

qg_random_shuffles_df = qg.copy()
contexts = []

for i, row in qg_random_shuffles_df.iterrows():
    if qg_random_shuffles[i].index(qg_gt_passage_table[i]) == 0:
        context = origin_wiki[str(qg_random_shuffles[i][0])]['text']
        for j in range(1, len(qg_random_shuffles[i])):
            context += ' ' + wiki[str(qg_random_shuffles[i][j])]['text']
        contexts.append(context)
    elif qg_random_shuffles[i].index(qg_gt_passage_table[i]) == 1:
        context = wiki[str(qg_random_shuffles[i][0])]['text']
        context += ' ' + origin_wiki[str(qg_random_shuffles[i][1])]['text']
        for j in range(2, len(qg_random_shuffles[i])):
            context += ' ' + wiki[str(qg_random_shuffles[i][j])]['text']
        contexts.append(context)
    elif qg_random_shuffles[i].index(qg_gt_passage_table[i]) == 2:
        context = wiki[str(qg_random_shuffles[i][0])]['text']
        context += ' ' + wiki[str(qg_random_shuffles[i][1])]['text']
        context += ' ' + origin_wiki[str(qg_random_shuffles[i][2])]['text']
        for j in range(3, len(qg_random_shuffles[i])):
            context += ' ' + wiki[str(qg_random_shuffles[i][j])]['text']
        contexts.append(context)
    elif qg_random_shuffles[i].index(qg_gt_passage_table[i]) == 3:
        context = wiki[str(qg_random_shuffles[i][0])]['text']
        context += ' ' + wiki[str(qg_random_shuffles[i][1])]['text']
        context += ' ' + wiki[str(qg_random_shuffles[i][2])]['text']
        context += ' ' + origin_wiki[str(qg_random_shuffles[i][3])]['text']
        for j in range(4, len(qg_random_shuffles[i])):
            context += ' ' + wiki[str(qg_random_shuffles[i][j])]['text']
        contexts.append(context)
    elif qg_random_shuffles[i].index(qg_gt_passage_table[i]) == 4:
        context = wiki[str(qg_random_shuffles[i][0])]['text']
        context += ' ' + wiki[str(qg_random_shuffles[i][1])]['text']
        context += ' ' + wiki[str(qg_random_shuffles[i][2])]['text']
        context += ' ' + wiki[str(qg_random_shuffles[i][3])]['text']
        context += ' ' + origin_wiki[str(qg_random_shuffles[i][4])]['text']
        contexts.append(context)

qg_random_shuffles_df['context'] = contexts
qg_random_shuffles_df['answers'] = qg_random_shuffles_answers.values()
qg_random_shuffles_df

IndexError: list index out of range

In [276]:
len(qg_random_shuffles_df)

17339